<a href="https://colab.research.google.com/github/b-fatma/S2I-DM/blob/master/src/3_merge/v2/merge_1_add_elevation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geopandas as gpd
import pandas as pd
import csv
import sys
import geopandas as gpd
import shapely

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading the Data

In [3]:
fire_landcover_path = '/content/drive/MyDrive/dm_fire_prediction/merge/fire_landcover.csv'
elevation_path = '/content/drive/MyDrive/dm_fire_prediction/premerge/elevation_no_binning.csv'

In [4]:
fl_df = pd.read_csv(fire_landcover_path)
el_df = pd.read_csv(elevation_path)

fl_df.head(), el_df.head()

(   longitude  latitude  fire                  geometry  lcccode_label
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0,
    longitude   latitude  elevation
 0   8.956111  37.556111        0.0
 1   8.947778  37.551944        0.0
 2   8.943611  37.539444        1.0
 3   8.947778  37.539444        0.0
 4   8.939444  37.535278        0.0)

In [5]:
fl_gdf = gpd.GeoDataFrame(fl_df, geometry=gpd.points_from_xy(fl_df.longitude, fl_df.latitude))
el_gdf = gpd.GeoDataFrame(el_df, geometry=gpd.points_from_xy(el_df.longitude, el_df.latitude))

fl_gdf.head(), el_gdf.head()

(   longitude  latitude  fire                  geometry  lcccode_label
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0,
    longitude   latitude  elevation                  geometry
 0   8.956111  37.556111        0.0  POINT (8.95611 37.55611)
 1   8.947778  37.551944        0.0  POINT (8.94778 37.55194)
 2   8.943611  37.539444        1.0  POINT (8.94361 37.53944)
 3   8.947778  37.539444        0.0  POINT (8.94778 37.53944)
 4   8.939444  37.535278        0.0  POINT (8.93944 37.53528))

## Merge

In [6]:
# Perform nearest-neighbor join
joined = fl_gdf.sjoin_nearest(
    el_gdf[['elevation', 'geometry']],
    how='left'
)

# If fl_gdf index structure must be preserved:
fl_gdf['elevation'] = joined['elevation']

fl_gdf.head(), joined.head()


(   longitude  latitude  fire                  geometry  lcccode_label  \
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0   
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0   
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0   
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0   
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0   
 
    elevation  
 0     1898.0  
 1      137.0  
 2      125.0  
 3      616.0  
 4      589.0  ,
    longitude  latitude  fire                  geometry  lcccode_label  \
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0   
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0   
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0   
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0   
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0   
 
    index_r

In [7]:
fl_gdf['elevation'].value_counts()

,count
elevation,
434.0,4015
431.0,3939
425.0,3924
432.0,3882
433.0,3841
...,...
2093.0,1
2065.0,1
2153.0,1


## Saving the Output

In [8]:
output_path = '/content/drive/MyDrive/dm_fire_prediction/merge/fire_landcover_elevation_no_binning.csv'

fl_gdf.drop(columns=['geometry']).to_csv(output_path, index=False)